### FITS images in python; image statistics and plots

This notebook provides examples of reading a FITS image into a numpy array, plotting a histogram of the measured count values, and determining basic statistics of the count distribution.

More information on handling FITS files in python can be found [here]([http://docs.astropy.org/en/stable/io/fits/).

In [ ]:
### for array operations
import numpy as np

### for plotting
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib inline

### for operations on FITS images
from astropy.io import fits

### statistics functions needed in this tutorial
from scipy import stats
from scipy.stats import norm

In [ ]:
### "fits.open" opens the FITS file
hdulist = fits.open('00000026.BIAS.FIT')

In [ ]:
### let's look at what's in it
### in this case, a single extention, i.e. it has index 0
hdulist.info()

In [ ]:
### an image 
header = hdulist[0].header

In [ ]:
### let's look at the header
header

In [ ]:
### ... and specifically the exposure time
header['EXPTIME']

In [ ]:
### let's look at the image data now
imagedata = hdulist[0].data

In [ ]:
### "imagedata" is now a numpy array
### to check it's dimensions, we can use:
imagedata.shape

In [ ]:
### for plotting the histogram, calculating statistics, etc., it's useful to convert the 2d array into a 1d list:
countvalues = imagedata.flatten()

In [ ]:
countvalues.shape

In [ ]:
### first, let's look at the maximum and minimum counts
print(np.max(countvalues))
print(np.min(countvalues))

In [ ]:
### plot a histogram, using a logarithmic y-axis
plt.hist(countvalues,bins=100);
plt.yscale('log')

In [ ]:
### let's re-plot, specifying a smaller range, and the number of bins:
plt.hist(countvalues,range=[900,1300], bins=100);
plt.yscale('log')

In [ ]:
### compute the mean, the median, the mode, and the standard deviation:
print(np.mean(countvalues))
print(stats.mode(countvalues)[0][0])
print(np.median(countvalues))
print(np.std(countvalues))

In [ ]:
### repeat the same, but clip "outliers":

clippedvalues = countvalues[countvalues<1050]

print(np.mean(clippedvalues))
print(stats.mode(clippedvalues)[0][0])
print(np.median(clippedvalues))
print(np.std(clippedvalues))

In [ ]:
### overplot a normal distribution, as specified by the mean and standard deviation

cmin=900
cmax=1200
nbins=100
normalization=(cmax-cmin)/nbins*len(countvalues[(countvalues>=cmin) & (countvalues<=cmax)])

clipmin=cmin
clipmax=1100
clippedvalues = countvalues[(countvalues>=clipmin) & (countvalues<=clipmax)]

mu=np.mean(clippedvalues)
sig=np.std(clippedvalues)
mode=stats.mode(clippedvalues)[0][0]

xarray=np.linspace(cmin,cmax,nbins*10)
yarray=normalization*norm.pdf(xarray,loc=mu, scale=sig)

plt.hist(countvalues,range=[cmin,cmax], bins=nbins);
plt.yscale('log')
plt.ylim([0.1,1e6])
plt.plot(xarray,yarray,color="red",linewidth=3.0)
plt.axvline(x=mode,linewidth=3.0,color="yellow")

### Operations on multiple files

In [ ]:
### open a second file file
hdu2 = fits.open('00000025.BIAS.FIT')

### read image data into numpy array
imagedata2 = hdu2[0].data

In [ ]:
### "simple" arithmetic operations such as addition etc. can be done straightforwardly such as
sum = imagedata + imagedata2

### for operations such as taking the mean, or median, of a list of array, 
### we first need to create an array out of all the images
### this can be done like this:
allimages=[imagedata2,imagedata]
### take the average of the two arrays
avg = np.mean(allimages,axis=0)

In [ ]:
### the result is another numpy array with the same shape:
avg.shape

In [ ]:
### write "sub" into a new fits file
newhdu = fits.PrimaryHDU(sub)
newhdu.writeto('avg.fits')